# atmodeller

## Tutorial 1: basic operation

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [1]:
from atmodeller import InteriorAtmosphereSystem, logger, FugacityConstraint, BufferedFugacityConstraint, SystemConstraint, MassConstraint
from atmodeller.thermodynamics import  NoSolubility,  StandardGibbsFreeEnergyOfFormationHollandAndPowell, StandardGibbsFreeEnergyOfFormationJANAF,  ChemicalComponent, GasSpecies
from atmodeller.solubilities import BasaltDixonCO2, BasaltS, PeridotiteH2O
from atmodeller.utilities import earth_oceans_to_kg

import logging

logger.setLevel(logging.DEBUG)

09:40:22 - atmodeller                     - INFO      - atmodeller version 0.1.0


### 1. Simple H2O-H2 system with prescribed H2O pressure

We define a list of the species we wish to include in the interior-atmosphere system and how they partition between the melt and the atmosphere, and the solid and the melt:

In [2]:
species: list[ChemicalComponent] = []
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))


09:40:22 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2O (H2O)
09:40:22 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2 (H2)


Although a choice is often made to constrain oxygen fugacity, we nevertheless need to explicitly include O2 as a species in the interior-atmosphere system:

In [3]:
species.append(GasSpecies(chemical_formula='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species

09:40:22 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: O2 (O2)


[GasSpecies(chemical_formula='H2O', common_name='H2O', formula=Formula('H2O'), solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x128583ed0>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='H2', common_name='H2', formula=Formula('H2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x132487f50>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='O2', common_name='O2', formula=Formula('O2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x132487910>, solid_melt_distribution_coefficient=0)]

We can then create an interior-atmosphere system using the list of species and (optionally) specifying the thermodynamic data to use. Note that this creates a planet with 'default properties' (a molten Earth). Adjusting the planet properties will be covered in a later tutorial.

In [4]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, gibbs_data=StandardGibbsFreeEnergyOfFormationJANAF())

09:40:22 - atmodeller.thermodynamics      - INFO      - Creating a new planet
09:40:22 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
09:40:22 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
09:40:22 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
09:40:22 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
09:40:22 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
09:40:22 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
09:40:22 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819973
09:40:22 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
09:40:22 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:40:22 - atmodeller.core                - INFO      - Species = ['H2',

Notice that for this simple system it has identified the single reaction that relates the 3 chosen species. To solve the system, we provide a constraint of the H2O pressure in bar:

In [5]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
constraints: list = [H2O_pressure]

A second constraint is required to close the system of equations, and often this is constraining the fugacity of some species at a buffered value, such as constraining the oxygen fugacity at the iron-wustite (IW) buffer. Constraining the fO2 at the IW buffer is the default if no arguments are provided:

In [6]:
O2_pressure: SystemConstraint = BufferedFugacityConstraint()
constraints.append(O2_pressure)
logger.info(constraints)

09:40:22 - atmodeller                     - INFO      - [FugacityConstraint(species='H2O', value=1, field='fugacity'), BufferedFugacityConstraint(species='O2', fugacity=<atmodeller.thermodynamics.IronWustiteBufferHirschmann object at 0x1321ae150>, log10_shift=0, field='fugacity')]


In [7]:
interior_atmosphere.solve(constraints)

09:40:22 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferHirschmann object at 0x1321ae150>,
                            log10_shift=0,
                            field='fugacity')]
09:40:22 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
09:40:22 - atmodeller.core                - DEBUG     - initial_log10_pressures = [1. 1. 1.]
09:40:22 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
09:40:22 - atmodeller.core                - INFO      - Row 00: Reaction 0: 1.0 H2 + 0.5 O2 = 1.0 H2O
09:40:22 - atmodeller.thermodynamics      - DEBUG     - Thermodynamic data for H2 (H2) = <thermochem.janaf.JanafPhase object 

{'H2': 0.978769527519816, 'O2': 8.700036057531159e-08, 'H2O': 1.0}

You can access the solution directly using:

In [8]:
interior_atmosphere.fugacities_dict

{'H2': 0.978769527519816, 'O2': 8.700036057531159e-08, 'H2O': 1.0}

You can access more information about the solution using:

In [9]:
interior_atmosphere.output

{'total_pressure_in_atmosphere': 1.9787696145201767,
 'mean_molar_mass_in_atmosphere': 0.010101415298632734,
 'H2': GasSpeciesOutput(mass_in_atmosphere=1.0145608083550336e+18, mass_in_solid=0.0, mass_in_melt=0.0, moles_in_atmosphere=5.0328382730488866e+20, moles_in_melt=0.0, moles_in_solid=0.0, ppmw_in_solid=0.0, ppmw_in_melt=0.0, pressure_in_atmosphere=0.978769527519816, volume_mixing_ratio=0.49463541401567035, mass_in_total=1.0145608083550336e+18),
 'O2': GasSpeciesOutput(mass_in_atmosphere=1431487022968.216, mass_in_solid=0.0, mass_in_melt=0.0, moles_in_atmosphere=44735633074111.695, moles_in_melt=0.0, moles_in_solid=0.0, ppmw_in_solid=0.0, ppmw_in_melt=0.0, pressure_in_atmosphere=8.700036057531159e-08, volume_mixing_ratio=4.396689737749381e-08, mass_in_total=1431487022968.216),
 'H2O': GasSpeciesOutput(mass_in_atmosphere=9.263470445724965e+18, mass_in_solid=0.0, mass_in_melt=2.205128880639838e+21, moles_in_atmosphere=5.142005478860794e+20, moles_in_melt=1.2240320571300573e+23, mole

### 2. System with C and H and prescribed pressures

We now extend the species list to additionally include C-species:

In [10]:
species: list[ChemicalComponent] = []
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='O2S', solubility=BasaltS(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='OS', solubility=BasaltS(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='S2', solubility=BasaltS(), solid_melt_distribution_coefficient=0))

species

09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2O (H2O)
09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2 (H2)
09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: O2 (O2)
09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CO (CO)
09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CO2 (CO2)
09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: O2S (O2S)
09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: OS (OS)
09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: S2 (S2)


[GasSpecies(chemical_formula='H2O', common_name='H2O', formula=Formula('H2O'), solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x13254d850>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='H2', common_name='H2', formula=Formula('H2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x13252ed50>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='O2', common_name='O2', formula=Formula('O2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x13252de10>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CO', common_name='CO', formula=Formula('CO'), solubility=<atmodeller.solubilities.NoSolubility object at 0x13252e3d0>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CO2', common_name='CO2', formula=Formula('CO2'), solubility=<atmodeller.solubilities.BasaltDixonCO2 object at 0x13254db90>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='O2S', common_nam

In [11]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)

09:40:23 - atmodeller.thermodynamics      - INFO      - Creating a new planet
09:40:23 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
09:40:23 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
09:40:23 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
09:40:23 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
09:40:23 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
09:40:23 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
09:40:23 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819973
09:40:23 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
09:40:23 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:40:23 - atmodeller.core                - INFO      - Species = ['CO',

Note now the system has identified two reactions in the network. With C present in the system we must provide at least 2 constraints, in addition to the oxygen fugacity:

In [12]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
CO2_pressure: SystemConstraint = FugacityConstraint(species='CO2', value=1)
S2_pressure: SystemConstraint = FugacityConstraint(species='S2', value=0.1)

O2_pressure: SystemConstraint = BufferedFugacityConstraint(log10_shift=3)
constraints: list[SystemConstraint] = [H2O_pressure, CO2_pressure, S2_pressure, O2_pressure]

In [13]:
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

09:40:23 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 FugacityConstraint(species='CO2', value=1, field='fugacity'),
 FugacityConstraint(species='S2', value=0.1, field='fugacity'),
 BufferedFugacityConstraint(species='O2',
                            fugacity=<atmodeller.thermodynamics.IronWustiteBufferHirschmann object at 0x13252e150>,
                            log10_shift=3,
                            field='fugacity')]
09:40:23 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
09:40:23 - atmodeller.core                - DEBUG     - initial_log10_pressures = [1. 1. 1. 1. 1. 1. 1. 1.]
09:40:23 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
09:40:23 - atmodeller.core                - INFO      - Row 00: Reaction 0: 2.0 OS = 

{'CO': 0.14111768417768383,
 'H2': 0.03095141011329474,
 'O2': 8.70003605753116e-05,
 'OS': 0.19032628149065683,
 'S2': 0.1,
 'CO2': 1.0,
 'H2O': 1.0,
 'O2S': 11.992168778729456}

There is not a requirement to necessarily impose the oxygen fugacity as a constraint. Instead, we can simply impose three pressure constraints (that span the reaction set) and allow for the oxygen fugacity to be solved. Note that if we do not specify an appropriate range of constraints we cannot solve the system of equations to give a unique solution and hence the code will raise an exception relating to a singular matrix.

In [14]:
H2O_pressure: SystemConstraint = FugacityConstraint(species='H2O', value=1)
H2_pressure: SystemConstraint = FugacityConstraint(species='H2', value=1)
CO_pressure: SystemConstraint = FugacityConstraint(species='CO', value=1)
S2_pressure: SystemConstraint = FugacityConstraint(species='S2', value=0.1)
constraints: list[SystemConstraint] = [H2O_pressure, H2_pressure, CO_pressure, S2_pressure]

In [15]:
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

09:40:33 - atmodeller.core                - INFO      - Constraints: [FugacityConstraint(species='H2O', value=1, field='fugacity'),
 FugacityConstraint(species='H2', value=1, field='fugacity'),
 FugacityConstraint(species='CO', value=1, field='fugacity'),
 FugacityConstraint(species='S2', value=0.1, field='fugacity')]
09:40:33 - atmodeller.core                - INFO      - Mixed pressure and mass constraints so attempting to solve a non-linear system of equations
09:40:33 - atmodeller.core                - DEBUG     - initial_log10_pressures = [1. 1. 1. 1. 1. 1. 1. 1.]
09:40:33 - atmodeller.core                - INFO      - The necessary number of constraints will be applied to the reaction network to solve the system
09:40:33 - atmodeller.core                - INFO      - Row 00: Reaction 0: 2.0 OS = 1.0 O2 + 1.0 S2
09:40:33 - atmodeller.thermodynamics      - DEBUG     - Thermodynamic data for CO (CO) = <thermochem.janaf.JanafPhase object at 0x1325521d0>
  Carbon Monoxide (CO)	C1O1(g)

{'CO': 1.0,
 'H2': 1.0,
 'O2': 8.334545698358514e-08,
 'OS': 0.005890866793755706,
 'S2': 0.1,
 'CO2': 0.2193304850037311,
 'H2O': 1.0,
 'O2S': 0.011488375226011623}

### 3. System with C and H and mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of pressure and mass constraints. We define the same species as before:

In [16]:
species: list[ChemicalComponent] = []
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
species

09:40:43 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2O (H2O)
09:40:43 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2 (H2)
09:40:43 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: O2 (O2)
09:40:43 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CO (CO)
09:40:43 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CO2 (CO2)


[GasSpecies(chemical_formula='H2O', common_name='H2O', formula=Formula('H2O'), solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x1325281d0>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='H2', common_name='H2', formula=Formula('H2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x13252a790>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='O2', common_name='O2', formula=Formula('O2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x13252bd90>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CO', common_name='CO', formula=Formula('CO'), solubility=<atmodeller.solubilities.NoSolubility object at 0x13252b1d0>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CO2', common_name='CO2', formula=Formula('CO2'), solubility=<atmodeller.solubilities.BasaltDixonCO2 object at 0x13252b850>, solid_melt_distribution_coefficient=0)]

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [17]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans)
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    BufferedFugacityConstraint()
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

09:40:43 - atmodeller.thermodynamics      - INFO      - Creating a new planet
09:40:43 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
09:40:43 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
09:40:43 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
09:40:43 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
09:40:43 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
09:40:43 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
09:40:43 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819973
09:40:43 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
09:40:43 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:40:43 - atmodeller.core                - INFO      - Species = ['CO',

{'CO': 59.53960886601458,
 'H2': 0.38489414856904475,
 'O2': 8.700036057531159e-08,
 'CO2': 13.342110601467452,
 'H2O': 0.3932428807263335}

### 4. Including more species such as CH4

It is straightforward to add more species to the system, although they must have their formation energies and masses already specified in the code.

In [18]:
species: list[ChemicalComponent] = []
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
species.append(GasSpecies(chemical_formula='CH4', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
species

09:40:49 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2O (H2O)
09:40:49 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: H2 (H2)
09:40:49 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: O2 (O2)
09:40:49 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CO (CO)
09:40:49 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CO2 (CO2)
09:40:49 - atmodeller.thermodynamics      - INFO      - Creating a GasSpecies: CH4 (CH4)


[GasSpecies(chemical_formula='H2O', common_name='H2O', formula=Formula('H2O'), solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x1325f2c50>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='H2', common_name='H2', formula=Formula('H2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x1325f0d90>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='O2', common_name='O2', formula=Formula('O2'), solubility=<atmodeller.solubilities.NoSolubility object at 0x1325f0550>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CO', common_name='CO', formula=Formula('CO'), solubility=<atmodeller.solubilities.NoSolubility object at 0x1325f0510>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CO2', common_name='CO2', formula=Formula('CO2'), solubility=<atmodeller.solubilities.BasaltDixonCO2 object at 0x1325d2d50>, solid_melt_distribution_coefficient=0),
 GasSpecies(chemical_formula='CH4', common_nam

We define a mixture of mass and oxygen fugacity constraints as before and solve the system. CH4 is not prevalent at 2000 K so the results are almost identical to those without CH4 presented above.

In [19]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans)
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    MassConstraint(species="H", value=mass_H),
    MassConstraint(species="C", value=mass_C),
    BufferedFugacityConstraint()
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species)
interior_atmosphere.solve(constraints)
interior_atmosphere.fugacities_dict

09:40:49 - atmodeller.thermodynamics      - INFO      - Creating a new planet
09:40:49 - atmodeller.thermodynamics      - INFO      - Mantle mass (kg) = 4208261222595110885130240.000000
09:40:49 - atmodeller.thermodynamics      - INFO      - Mantle melt fraction = 1.000000
09:40:49 - atmodeller.thermodynamics      - INFO      - Core mass fraction = 0.295335
09:40:49 - atmodeller.thermodynamics      - INFO      - Planetary radius (m) = 6371000.000000
09:40:49 - atmodeller.thermodynamics      - INFO      - Planetary mass (kg) = 5972000000000000327155712.000000
09:40:49 - atmodeller.thermodynamics      - INFO      - Surface temperature (K) = 2000.000000
09:40:49 - atmodeller.thermodynamics      - INFO      - Surface gravity (m/s^2) = 9.819973
09:40:49 - atmodeller.thermodynamics      - INFO      - Melt Composition = None
09:40:49 - atmodeller.core                - INFO      - Creating a new interior-atmosphere system
09:40:49 - atmodeller.core                - INFO      - Species = ['CO',

{'CO': 59.53960841803111,
 'H2': 0.38489414725265664,
 'O2': 8.700036057531159e-08,
 'CO2': 13.342110501079746,
 'H2O': 0.39324287938139174,
 'CH4': 3.8841985669277415e-07}